# Sesión 7: Más Allá de lo Relacional - La Filosofía Multi-Modelo

**Objetivo:** Comprender las limitaciones del modelo relacional estricto al enfrentarnos a datos complejos y semi-estructurados como las noticias. En este cuaderno, exploraremos la filosofía detrás de las bases de datos NoSQL y multi-modelo, usando Azure Cosmos DB como nuestro caso de estudio teórico, para justificar la necesidad de las arquitecturas que construiremos en Databricks.

**Dataset:** `Noticias.csv`

## El Anti-Patrón - Intentando Forzar un Modelo Relacional

Imaginemos que queremos almacenar nuestro dataset de noticias en una base de datos relacional tradicional, siguiendo las reglas de normalización (3NF). Para evitar la redundancia y mantener la integridad, tendríamos que crear un esquema complejo y fragmentado.

Necesitaríamos, como mínimo, las siguientes tablas:
* **`noticias`**: Contendría el ID, título, enlace y contenido principal.
* **`etiquetas`**: Una tabla para los tipos de noticias (Justicia, Deportes, etc.).
* **`noticia_etiqueta`**: Una tabla de unión para vincular una noticia con su etiqueta.
* **`entidades`**: Una tabla para almacenar cada persona, organización o lugar único mencionado.
* **`noticia_entidad`**: Otra tabla de unión para registrar qué entidades aparecen en qué noticias.

El resultado es que la información de **una sola noticia** queda esparcida por al menos 5 tablas diferentes.

In [0]:
# A continuación, un ejemplo conceptual de la consulta SQL que necesitaríamos
# para reconstruir la información completa de UNA SOLA noticia.

# (Este código es una simulación para ilustrar el punto, no para ser ejecutado)

hypothetical_sql_query = """
SELECT
  n.titulo,
  n.contenido,
  e.nombre_etiqueta,
  ent.nombre_entidad,
  ent.tipo_entidad
FROM noticias AS n
JOIN noticia_etiqueta AS ne ON n.id_noticia = ne.id_noticia
JOIN etiquetas AS e ON ne.id_etiqueta = e.id_etiqueta
JOIN noticia_entidad AS n_ent ON n.id_noticia = n_ent.id_noticia
JOIN entidades AS ent ON n_ent.id_entidad = ent.id_entidad
WHERE n.id_noticia = 'CMS-5171887';
"""

print("--- Consulta SQL para obtener una noticia completa ---")
print(hypothetical_sql_query)
print("\n# Observación: Se necesitan 4 JOINs solo para obtener la información básica.")
print("# ¿Y si quisiéramos todas las noticias de una entidad? La complejidad aumenta.")

## La Filosofía de Cosmos DB - "El Modelo Correcto para el Trabajo Correcto"

El problema anterior no significa que el modelo relacional sea malo; significa que no es la herramienta óptima para **este tipo de datos**.

Azure Cosmos DB introduce una filosofía diferente: **ser multi-modelo**. En lugar de forzar los datos a una estructura de tablas, nos ofrece diferentes "APIs" o "personalidades" para que elijamos la que mejor se adapte a nuestro problema.

* **¿Tus datos se parecen a un documento JSON?** Usa la API for NoSQL.
* **¿Tus datos son una red de conexiones?** Usa la API for Gremlin (Grafos).
* **¿Son datos de series temporales?** Podrías usar la API para Cassandra (Columnar).

La lección clave es: **el diseño de la base de datos debe seguir la forma de los datos, y no al revés.**

## Profundizando en el Modelo de Documentos

El modelo más intuitivo para nuestras noticias es el **documento**. La idea es simple: toda la información relevante a una sola noticia se almacena junta en una única estructura flexible, similar a un JSON.

**Ventajas:**
1.  **Rendimiento de Lectura:** Para obtener una noticia, se realiza una sola operación de lectura. No hay `JOINs`. Esto es órdenes de magnitud más rápido.
2.  **Flexibilidad de Esquema:** Si una noticia tiene un campo adicional (ej. `subtitulo`) y otra no, no hay problema. No hay que alterar una tabla rígida.
3.  **Alineación con el Desarrollo:** Los desarrolladores de aplicaciones suelen trabajar con objetos (JSON). Un modelo de documentos elimina la fricción de tener que "traducir" entre objetos y tablas relacionales.

In [0]:
import pandas as pd

# Ruta del archivo en Databricks Volumes
file_path = "/Volumes/sesion_5/bronze/raw_files/Noticias.xlsx"

# pandas
df_noticias = pd.read_excel(file_path)


In [0]:
import csv 
df_noticias.to_csv('Noticias.csv', index=False, quoting=csv.QUOTE_ALL)


In [0]:
## Visualizando una Noticia como un Documento (Versión Corregida para Databricks)

# Vamos a leer nuestro CSV desde el volumen de Databricks usando PySpark
# y a convertir la primera fila en un diccionario para simular cómo se vería
# un documento JSON en Cosmos DB o MongoDB.

import json

# Ruta del archivo en Databricks Volumes
file_path = "/Volumes/sesion_5/bronze/raw_files/Noticias.csv"

# Leemos el CSV usando el lector nativo de Spark.
# - "header=True" le dice a Spark que la primera fila es el encabezado.
# - "inferSchema=True" intenta adivinar los tipos de datos de cada columna.
df_noticias = spark.read.csv(file_path, header=True, inferSchema=True)

# Tomamos la primera fila del DataFrame de Spark.
# .first() devuelve un objeto Row.
primera_noticia_row = df_noticias.first()

# Convertimos el objeto Row a un diccionario de Python.
primera_noticia_dict = primera_noticia_row.asDict()

# Convertimos el diccionario a una cadena JSON con formato para mejor lectura.
primera_noticia_json = json.dumps(primera_noticia_dict, indent=4, ensure_ascii=False)

print("--- Una fila del CSV representada como un Documento JSON ---")
print(primera_noticia_json)

## Profundizando en el Modelo de Grafos

¿Y si nuestra pregunta no es sobre el contenido de *una* noticia, sino sobre las **conexiones ocultas *entre* las noticias**?

* *"¿Qué políticos han opinado sobre las actividades de la empresa X en los últimos 3 meses?"*
* *"Muéstrame la red de colaboración entre la 'Sijín' y el 'Gaula' según los operativos reportados."*

Responder a estas preguntas con SQL requeriría `JOINs` recursivos y sería extremadamente lento e ineficiente. El **modelo de grafos** está diseñado específicamente para esto. Se enfoca en dos cosas: **entidades (nodos)** y las **relaciones (aristas)** que las conectan.

In [0]:
# Conceptualizando el Grafo de Noticias

# Imaginemos que extraemos las entidades de la primera noticia.
# No necesitamos código complejo de NLP ahora, solo entender la lógica.

contenido = primera_noticia_dict['contenido']

# Entidades (Nodos) que podríamos extraer:
nodos = [
    {"id": 1, "label": "Coronel Jorge Iván Flórez", "tipo": "Persona"},
    {"id": 2, "label": "Policía Metropolitana de Cúcuta", "tipo": "Organización"},
    {"id": 3, "label": "Los Patios", "tipo": "Lugar"},
    {"id": 4, "label": "Sijín", "tipo": "Organización"},
    {"id": 5, "label": "Gaula", "tipo": "Organización"}
]

# Relaciones (Aristas) que podríamos inferir:
aristas = [
    {"source": 1, "target": 2, "label": "ES_COMANDANTE_DE"},
    {"source": 2, "target": 3, "label": "REALIZÓ_OPERATIVO_EN"},
    {"source": 4, "target": 2, "label": "ES_PARTE_DE"},
    {"source": 5, "target": 2, "label": "ES_PARTE_DE"}
]

print("--- Nodos Extraídos ---")
for nodo in nodos:
    print(nodo)

print("\n--- Aristas Inferidas ---")
for arista in aristas:
    print(arista)

## Un Avance del Futuro - El Modelo Vectorial

Hasta ahora, hemos hablado de la estructura (documentos) y las entidades explícitas (grafos). Pero, ¿qué hay del **significado** o la **semántica** del contenido?

* ¿Cómo encontramos noticias que hablen de "inestabilidad económica" si no usan esas palabras exactas, sino que hablan de "inflación", "desempleo" y "devaluación"?

Aquí es donde entra el **modelo vectorial**, la base de la IA generativa y la búsqueda semántica. La idea es convertir cada `contenido` de noticia en un vector numérico (un "embedding") que represente su significado.

Hablaremos de esto en detalle más adelante, pero es el tercer pilar de una arquitectura de datos moderna para contenido no estructurado.

## Conclusión y Nuestro Camino en Databricks

Hemos establecido la filosofía: para datos complejos como las noticias, un solo modelo (el relacional) no es suficiente. Necesitamos una caja de herramientas más grande.

En las próximas secciones de este taller, tomaremos esta filosofía multi-modelo y la implementaremos de manera práctica y a gran escala, no en Cosmos DB, sino utilizando el poder unificado de **Databricks**:

1.  **Implementaremos el Modelo de Documentos** usando tablas Delta con columnas `STRUCT`.
2.  **Construiremos y analizaremos un Grafo de Conocimiento** con PySpark y GraphFrames.
3.  **Crearemos un motor de Búsqueda Semántica** generando embeddings y usando Databricks Vector Search.

¡Manos a la obra!